# Benchmarking

ATOM tries to predict the paleo climate in time slices of million years. The atmospheric and the hydrospheric circulations are computed based on the laminar and steady Navier-Stokes equations with additional transport equations for the water vapour, cloud water, cloud ice resulting in precipitation predictions and for the salinity transport. The ocean surface water velocity distribution results from the surface wind velocity distribution of the atmospheric circulations. 

The code is written for laptop usage, which makes approximations necessary, one of which is the assumption of a laminar and steady flow environment. The application of the 4th order Runge-Kutta solution scheme including immersed boundary conditions and the assumption of a sperical orthogonal coordinate system makes the code fast.

ATOM's main fluid dynamical features, Atmosphere and Hydrosphere, are permanently in development. Changements are uploaded to GitHub.

This code allows the vizualisation of the results for time slices the user prescribes. For the modern world the results are comparable to NASA measurements.


In [ ]:
import sys
import matplotlib.pyplot as plt
import os.path
import numpy as np
import shutil
import pyatom

from mpl_toolkits.basemap import Basemap
from pyatom import Model

from numpy import linspace
from numpy import meshgrid

model = Model()

# for "a_h = atm" only the atmospheric circulation used:
#a_h = 'atmosphere'
#a_h = 'hydrosphere'     # the transfer file with the extension *.vw from the corresponding atm-run must be available!
a_h = 'atmosphere and hydrosphere'

# for "rebuild = True" old output files are deleted:
rebuild = False

# applied configuration files for atmosphere and/or hydrosphere are named:
config_xml_atm = 'config_atm.xml'
config_xml_hyd = 'config_hyd.xml'

# time slices in Ma to be compared are named:
#time_slice = [ 0, 14 ]
#time_slice = [ 0 ]
time_slice = [ 14 ]

# applied time slices
for t_s in time_slice:
    print 'time slice in Ma = ', t_s
            
    # naming the output paths
    output_path = 'output-%s/' % t_s
    print 'output path = ', output_path
            
    # deleting old output paths in case "rebuild = True"
    if rebuild:
        shutil.rmtree ( output_path )

    if a_h == 'atmosphere':
        # runing the atmosphere model
        print 'run model: time slice in Ma = ', t_s, '   output path = ', output_path, '   configuration file = ', config_xml_atm
        model.run_atm( t_s, output_path, config_xml_atm )
    elif a_h == 'hydrosphere':        
        # runing the hydrosphere model
        print 'run model: time slice in Ma = ', t_s, '   output path = ', output_path, '   configuration file = ', config_xml_hyd
        model.run_hyd( t_s, output_path, config_xml_hyd )
    elif a_h == 'atmosphere and hydrosphere':        
        # runing the atmosphere model
        print 'run model: time slice in Ma = ', t_s, '   output path = ', output_path, '   configuration file = ', config_xml_atm
        model.run_atm( t_s, output_path, config_xml_atm )
        # runing the hydrosphere model
        print 'run model: time slice in Ma = ', t_s, '   output path = ', output_path, '   configuration file = ', config_xml_hyd
        model.run_hyd( t_s, output_path, config_xml_hyd )

print 'end of benchmark session running %s' % a_h


In [ ]:
import os.path
import numpy as np
Ma = t_s
datapath = 'output-%s/' % Ma

atm_header = ['lats (deg)', 'lons (deg)', 'topography', 'v-velocity (m/s)', 'w-velocity (m/s)', 'velocity-mag (m/s)', 'temperature (Celcius)', 'water_vapour (g/kg)', 'precipitation (mm)', 'precipitable_water (mm)']
hyd_header = ['lats (deg)', 'lons (deg)', 'topography', 'v-velocity (m/s)', 'w-velocity (m/s)', 'velocity-mag (m/s)', 'temperature (Celsius)', 'salinity (psu)', 'bottom_water (m/s)', 'upwelling (m/s)', 'downwelling (m/s)']

adata = np.loadtxt(os.path.join(datapath, '[%sMa_Golonka.xyz]_PlotData_Atm.xyz' % Ma), skiprows=1)
hdata = np.loadtxt(os.path.join(datapath, '[%sMa_Golonka.xyz]_PlotData_Hyd.xyz' % Ma), skiprows=1)


In [ ]:
import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap
import numpy as np
from numpy import linspace
from numpy import meshgrid

def plot_xyz(data_topo, series, title, lats, lons):
    plt.figure(figsize=(15, 8))

    m = Basemap(llcrnrlon=0,llcrnrlat=-90,urcrnrlon=360,urcrnrlat=90,projection='kav7', lon_0=0)
    
    xi, yi = m(lons, lats)

    #print 'lons = ', lons.shape, lons
    #print 'lats = ', lats.shape, lats
    #print 'series = ', series.shape, series
    #print 'xi = ', xi.shape, xi
    #print 'yi = ', yi.shape, yi
    
    cs = m.scatter(xi, yi, marker='.', c=series, alpha=0.5, lw=0)
    
    # Add Grid Lines
    m.drawparallels(np.arange(-90., 90., 10.), labels=[1,0,0,0], fontsize=10)
    m.drawmeridians(np.arange(-180., 180., 10.), labels=[0,0,0,1], fontsize=10)

    if Ma == 0:
        m.drawcoastlines()
    else:  
        x_d = np.zeros((361))
        y_d = np.zeros((181))
        series_draw = np.zeros((361,181))

        i = 0
        for j in range (0, 65341, 181):
            x_d[i] = xi[j]
            i = i + 1

        for j in range (0, 181):        
            y_d[j] = yi[j]
                
        y_draw, x_draw = np.meshgrid( y_d, x_d )
        #print 'x_draw = ', x_draw.shape, x_draw
        #print 'y_draw = ', y_draw.shape, y_draw
  
        for i in range (0, 361):
            for j in range (0, 181):
                series_draw[i][j] = data_topo[j]
        #print 'series_draw.shape = ', series_draw.shape, series_draw
        
        con = m.contour( x_draw, y_draw, series_draw )
        
    # Add Colorbar
    cbar = m.colorbar(cs, location='bottom', pad="10%")
    # cbar.set_label(tmax_units)

    # Add Title
    plt.title(title)
    print plt.title(title)

    plt.show()

# Plot all Atmosphere outputs

The outputs are given for example in `output-0/[0Ma_Golonka.xyz]_PlotData_Atm.xyz` files. 

Additional results must be prepared in the PostProcess_Atm.cpp in the class method  PostProcess_Atmosphere::Atmosphere_PlotData()


In [ ]:
# Plot all of Atmosphere outputs

alats = -(adata[:, 1] - 90.)
alons = adata[:, 0]
data_topo = adata[:, 2]

title = atm_header[ 2 ]

for index in range(3, 6):
    title = atm_header[index]
    plot_xyz(data_topo, adata[:, index], title + '  ATOM atmosphere at Ma=%s' % Ma, alats, alons)


In [ ]:
for index in range(6, len(atm_header)):
    title = atm_header[index]
    plot_xyz(data_topo, adata[:, index], title + '  ATOM atmosphere at Ma=%s' % Ma, alats, alons)



# Plot all Hydrosphere outputs

The outputs are given for example in `output-0/[0Ma_Golonka.xyz]_PlotData_Hyd.xyz` files. 

Additional results must be prepared in the PostProcess_Hyd.cpp in the class method  PostProcess_Hydrosphere::Hydrosphere_PlotData()

In [ ]:
# Plot all of Hydrosphere outputs
hlats = -(hdata[:, 0] - 90)
hlons = hdata[:, 1]
data_topo = adata[:, 2]

title = hyd_header[ 2 ]

for index in range(3, 6):
    title = hyd_header[index]
    plot_xyz(data_topo, hdata[:, index], title + '  ATOM hydrosphere at Ma=%s' % Ma, hlats, hlons)


In [ ]:
for index in range(6, len(hyd_header)):
    title = hyd_header[index]
    plot_xyz(data_topo, hdata[:, index], title + '  ATOM hydrosphere at Ma=%s' % Ma, hlats, hlons)


In [ ]:
for index in range(9, len(hyd_header)):
    title = hyd_header[index]
    plot_xyz(data_topo, hdata[:, index], title + '  ATOM hydrosphere at Ma=%s' % Ma, hlats, hlons)



# Compare surface precipitation

ATOM's precipitation for the modern world can be compared to NASA's measured precipitation.


In [ ]:
# nsp: NASA Surface Precipitation
nsp = np.genfromtxt('../data/SurfacePrecipitation_NASA.xyz')
# columns lons, lats, precip

if Ma == 0:
    plot_xyz(data_topo, lons=nsp[:, 0], lats=nsp[:, 1], series=nsp[:, 2], title='NASA Surface Precipitation')

plot_xyz(data_topo, adata[:, 7], 'ATOM precipitation at Ma=%s' % Ma, alats, alons)



# Compare surface temperature

ATOM's temperature distribution for the modern world can be compared to NASA's measured precipitation.
ATOM also can start with NASA's measured temperature distribution as initial condition. This must be changed in cAtmosphereModel.cpp.



In [ ]:
# st: Surface Temperature
st = np.genfromtxt('../data/SurfaceTemperature_NASA.xyz')
# columns lons, lats, ?

if Ma == 0:
    plot_xyz(data_topo, lons=st[:, 0], lats=st[:, 1], series=st[:, 2], title='Surface Temperature_NASA')
plot_xyz(data_topo, adata[:, 6], 'ATOM atmosphere temperature (Celsius) at Ma=%s' % Ma, alats, alons)
plot_xyz(data_topo, hdata[:, 6], 'ATOM hydrosphere temperature (Celsius) at Ma=%s' % Ma, hlats, hlons)


# Surface Winds benchmarking

Surface winds benchmarking is still in progress.
Reference data comes from https://podaac.jpl.nasa.gov/dataset/QSCAT_L3_SFC_WIND_SPEED_1DEG_1MO
     

In [ ]:
# load windspeed data
from netCDF4 import Dataset
root = Dataset('windspeed/sfcWind_QuikSCAT_L2B_v20110531_199908-200910.nc')
# print root
# print root.dimensions.keys()
#print root.variables.keys()

t = root.variables['time'][-1]  # just take the newest timeslice

# NOTE: wind mags vary substantially with time; what time should we be using? or average it all out?

swlats = root.variables['lat'][:]
swlons = root.variables['lon'][:]
swspeed = root.variables['sfcWind'][-1]

lonv, latv = np.meshgrid(swlons, swlats)

if Ma == 0:
    plot_xyz(data_topo, lats=latv.flatten(), lons=lonv.flatten(), title='Surface Wind QuikSCAT', series=swspeed.flatten())

root.close()

In [ ]:
# load windspeed data
import netCDF4
from netCDF4 import Dataset
root = Dataset('windspeed/uas_QuikSCAT_L2B_v20110531_199908-200910.nc')
#print root.variables.keys()

t = root.variables['time'][-1]  # just take the newest timeslice

# NOTE: wind mags vary substantially with time; what time should we be using? or average it all out?

swlats = root.variables['lat'][:]
swlons = root.variables['lon'][:]
swspeed = root.variables['uas'][-1]

lonv, latv = np.meshgrid(swlons, swlats)

if Ma == 0:
    plot_xyz(data_topo, lats=latv.flatten(), lons=lonv.flatten(), title='Surface Wind QuikSCAT u-mag', series=swspeed.flatten())

root.close()

In [ ]:
# load windspeed data
from netCDF4 import Dataset
root = Dataset('windspeed/vas_QuikSCAT_L2B_v20110531_199908-200910.nc')
#print root.variables.keys()

t = root.variables['time'][-1]  # just take the newest timeslice

# NOTE: wind mags vary substantially with time; what time should we be using? or average it all out?

swlats = root.variables['lat'][:]
swlons = root.variables['lon'][:]
swspeed = root.variables['vas'][-1]

lonv, latv = np.meshgrid(swlons, swlats)

if Ma == 0:
    plot_xyz(data_topo, lats=latv.flatten(), lons=lonv.flatten(), title='Surface Wind QuikSCAT v-mag', series=swspeed.flatten())

root.close()

In [ ]:
for index in range(3, 6):
#    title = 'Atmosphere: %s' % atm_header[index]
    title = atm_header[index]
    plot_xyz(data_topo, adata[:, index], title + '  ATOM atmosphere at Ma=%s' % Ma, alats, alons)



# Surface Currents benchmarking

Surface currents benchmarking is still in progress.
Using data from https://podaac.jpl.nasa.gov/dataset/OSCAR_L4_OC_third-deg


In [ ]:
from netCDF4 import Dataset
root = Dataset('oscar/oscar_vel8879.nc')
#print root.variables.keys()

sclats = root.variables['latitude'][:]
sclons = root.variables['longitude'][:]
u = root.variables['u'][0][0][:]
v = root.variables['v'][0][0][:]


#print sclats.shape 
#print sclons.shape
#print u.shape
#print v.shape

lonv, latv = np.meshgrid(sclons, sclats)

if Ma == 0:
    plot_xyz(data_topo, lats=latv.flatten(), lons=lonv.flatten(), title='OSCAR Surface Currents u-mag', series=np.log(u.flatten()))


In [ ]:
if Ma == 0:
    plot_xyz(data_topo, lats=latv.flatten(), lons=lonv.flatten(), title='OSCAR Surface Currents v-mag', series=v.flatten())

root.close()

In [ ]:
# NOTE: mysterious transpositions
hlats = -(hdata[:, 0] - 90)
hlons = hdata[:, 1]

for index in range(3, 6):
#    title = "Hydrosphere: %s" % hyd_header[index]
    title = hyd_header[index]
    plot_xyz(data_topo, hdata[:, index], title + '  ATOM hydrosphere at Ma=%s' % Ma, hlats, hlons)
